## 챕터 11: 인터페이스: 프로토콜에서 ABC까지

### 파이썬 문화에서의 인터페이스와 프로토콜
- 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스를 스몰토크에서는 **프로토콜**이라고 불렀다. 프로토콜은 상속과 무관하다. 

- 프로토콜은 인터페이스지만 비공식적이다. 즉, 문서와 관례에 따라 정의되지만, 공식 인터페이스처럼 강제할 수 없다.

- 시퀀스 프로토콜은 파이썬에서 가장 핵심적인 인터페이스 중 하나다.

### 파이썬은 시퀀스를 찾아낸다.

In [2]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()
f[1]

10

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

In [5]:
15 in f

False

- _iter_() 메서드는 아직 구현하지 않았지만, 대체 수단인 _getitem_() 메서드가 구현되어 있으므로 Foo 객체를 반복할 수 있다. 파이썬 인터프리터는 0부터 시작하는 정수 인덱스로 _getitem_() 메서드를 호출하여 객체 반복을 시도하기 때문이다. 파이썬은 Foo 객체를 반복할 만큼 충분히 똑똑하므로 Foo에 _contains_() 메서드가 구현되어 있지 않더라도, 객체 전체를 조사해서 항목을 찾아냄으로써 in 연산자도 작동시킬 수 있다.

### 런타임에 프로토콜을 구현하는 멍키 패칭
- randomshuffle() 함수

In [7]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[2, 3, 6, 1, 8, 5, 0, 4, 9, 7]

In [8]:
from random import shuffle
from frenchdeck import FrenchDeck
deck = FrenchDeck()
shuffle(deck)

ModuleNotFoundError: No module named 'frenchdeck'

- shuffle () 함수는 컬렉션 안의 항목들을 교환시킴으로써 작동하는데, FrenchDeck 클래스는 **불변** 시퀀스 프로토콜만 구현하고 있다. 가변 시퀀스는 _setitem_() 메서드도 지원해야 한다.

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card # 그 함수를 FrenchDeck 클래스의 __setitem__이라는 이름의 속성에 할당한다.
shuffle(deck) #이제 FrenchDeck 클래스가 가변 시퀀스 프로토콜에 필요한 메서드를 구현하므로, deck을 섞을 수 있다.
deck[:5]

NameError: name 'FrenchDeck' is not defined

- deck 객체에 _cards 라는 이름의 속성이 있고, _cards가 가변 시퀀스임을 set_card()가 알고 있다는 것이 비결이다. 그러고 나서 set_card() 함수가 FrenchDeck 클래스의 _setitem_ 특별 메서드에 연결된다. 이 방법은 **멍키 패칭**의 한 예다. 멍키 패칭은 소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위를 말한다. 멍키 패칭은 강력하지만, 비공개 속성이나 문서화되지 않은 부분을 다루는 경우가 많기 때문에 패치하는 코드와 패치될 프로그램이 아주 밀접하게 연관되어 있다.

### 알렉스 마르텔리의 물새
- 클래스를 ABC의 서브클래스로 인식시키기 위해 등록할 필요가 없는 경우도 있는데, 이런 경우 ABC는 결국 몇 가지 특별 메서드로 축약된다.

In [11]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

- abc.Sized 클래스는 Struggle을 '일종의 서브클래스'로 인식한다. 
- isinstance(the_arg, collections.abc.Sequence)

- field_names는 공백이나 콤마로 구분된 식별자들의 문자열이나 식별자들의 시퀀스를 받는다.

In [12]:
# 문자열이나 문자열의 반복 가능형을 처리하기 위한 덕 타이핑
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)

NameError: name 'field_names' is not defined

**ABC는 '시퀀스'나 '정확한 숫자' 같은 일종의 프레임워크가 소개하는 상당히 광범위한 개념이나 추상성을 담기 위한 것이다.**

### ABC 상속하기

In [13]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.rank]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

/Users/ahrimahn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """


- FrenchDeck2는 Sequence로부터 _contains_(), _iter_(), _reversed_(), index(), count()와 같은 바로 사용할 수 있는 메서드를 상속한다. MutableSequence 클래스로부터는 append(), reverse(), extend(), pop(), remove(), _iadd_() 메서드를 상속한다.


### 표준 라이브러리의 ABC
* Iterablle, Container, Sized: 모든 컬렉션은 이 ABC를 상속하거나, 적어도 호환되는 프로토콜을 구현해야 한다. Iterable은 _iter_()를 통해 반복을, Container는 _contains_()를 통해 in연산자를, Sized는 _len_()를 통해 len()메서드를 지원한다.

* Sequence, Mapping, Set: 주요 불변 컬렉션형으로서, 각기 가변형 서브클래스가 있다. 

* MappingView: items(), keys(), values(), 매서드에서 반환된 객체는 각기 ItemsView, KeysView, ValuesView를 상속한다.

* Callable, Hashable: collections.abc가 파이썬 표준 라이브러리 안에서 ABC를 정의한 최초의 패키지이다.

* Iterator: Iterable을 상속한다.

### ABC의 숫자탑
* 정수형인지 검사해야 하는 경우 isinstance(x, numbers.Integral)을 이용해서 int형, bool형, 또는 자신을 numbers ABC에 등록한 정수형을 받을 수 있다. 그리고 언제든 클래스를 numbers.Integral의 가상 서브클래스로 등록하면, 해당 클래스의 객체가 isinstance(x, numbers.Integral) 검사를 통과할 수 있다.


### ABC의 정의와 사용
* Tombola ABC는 메서드를 네 개 가지고 있다. 그 중 두 개의 추상 메서드는 다음과 같다.
    * load(): 항목을 컨테이너 안에 넣는다.
    * pick(): 컨테이너 안에서 무작위로 항목 하나를 꺼내서 반환한다.
* 나머지 두 개
    * loaded(): 컨테이너 안에 항목이 하나 이상 들어 있으면 True 반환
    * inspect(): 내용물을 변경하지 않고 현재 컨테이너 안에 들어 있는 항목으로부터 만든 정렬된 튜플을 반환한다.

In [1]:
import abc

class Tombola(abc, ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
        
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다."""
        
        
    def loaded(self):
        """최소 한 개의 항목들로 구성된 정렬된 튜플을 반환한다."""
        return bool(self.isinstance())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플을 반환한다."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        delf.load(items)
        return tuple(sorted(items))

NameError: name 'ABC' is not defined

### ABC 상세 구문

#### Tombola ABC 상속하기

In [2]:
import random

from tombola import Tombola

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from rmpty BingoCage')
            
            
    def __call__(self):
        self.pick()

ModuleNotFoundError: No module named 'tombola'

In [3]:
import random

from tombola import Tombola

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

ModuleNotFoundError: No module named 'tombola'

### Tombola의 가상 서브클래스

In [4]:
from random import randrange

from tombola import Tombola

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
# Tombola.register(TomboList)

ModuleNotFoundError: No module named 'tombola'

In [5]:
from tombola import Tombola
from tombolist import TomboList
issubclass(TomboList, Tombola)

ModuleNotFoundError: No module named 'tombola'

In [6]:
t = TomboList(range(100))
isinstance(t, Tombola)

NameError: name 'TomboList' is not defined

### Tombola 서브클래스 테스트 방법
* _subclasses_(): 클래스의 바로 아래 서브클래스의 리스트를 반환하는 메서드, 리스트에 가상 서브클래스는 들어가지 않는다.
* _abc_registry: ABC에서만 사용할 수 있는 데이터 속성으로, 추상 클래스의 등록된 가상 서브클래스에 대한 약한 참조를 담고 있는 WeakSet이다.

In [7]:
import doctest

from tombola import Tombola

# 테스트할 모듈
import bingo, lotto, tombolist, drum

TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

def main(argv):
    verbose = '-v' in argv
    real_subclasses = Tombola.__subclasses__()
    virtual_subclasses = list(Tombola._abc_registry)
    
    for cls in real_subclasses + virtual_subclasses:
        test(cls, verbose)

def test(cls, verbose=False):
    
    res = doctest.testfile(TEST_FILE,
                          globs={'ConcreteTombola':cls},
                          verbose=verbose,
                          optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG, format(cls.__name__, res, tag))
    
if __name__ == '__main__':
    import sys
    main(sys.argv)

ModuleNotFoundError: No module named 'tombola'

### register()의 실제 용법

### 오리처럼 행동할 수 있는 거위

In [8]:
class Struggle:
    def __len__(self): return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [9]:
issubclass(Struggle, abc.Sized)

True

In [ ]:
class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                # C.__mro__에 나열된 클래스 중 __dict__ 속성에 __len__이라는 속성을 가진 클래스가 하나라도 있으면
                return True # True를 반환해서 C가 Sized의 가상 서브클래스임을 알려준다.
        return NotImplemented # 그렇지 않으면 NotImplemented를 반환해서 서브클래스 검사를 진행할 수 있게 한다.